In [ ]:
import ROOT, shutil
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
pwd = ROOT.gSystem.pwd()

# sPlot with Gaussian signal and Polynomial background

First, we must create a new, weighted phi asymmetry observable fit using "RooFitSkeleton". This is accomplished by first **make-ing** the brufit repository using JLab's Jupyterhub (not via ssh). Then, you must 'cd' into the macros directory, run **root**, and execute the line **.L LoadBru.C**. Then, you can safely run the necessary commands to generate the asymmetry observable. See (https://github.com/dglazier/brufit/tree/dev/tutorials/WeightedObservable) for more details on that.

Create the sPlot fit manager and set the ouput directory for fit results, plots and weights

In [ ]:
splot = ROOT.sPlot()
try:
    shutil.rmtree(pwd+"/outSplot")
except OSError as e:
    print(e)
splot.SetUp().SetOutDir(pwd+"/outSplot")
splot.SetRedirectOutput() # Forces minimal output when fitting diphoton peaks

Load up the diphoton mass fitting variable, the exact variable responsible for identifying background from signal

In [ ]:
splot.SetUp().LoadVariable("Mdiphoton[0.08,0.22]");
splot.SetUp().SetIDBranchName("fidmerge");

Create the basic signal for the diphoton mass

In [ ]:
splot.SetUp().FactoryPDF("Gaussian::Signal(Mdiphoton,#mu[0.131,0.129,0.136],#sigma[0.1,0.001,0.2])")
splot.SetUp().LoadSpeciesPDF("Signal")

Create the basic background fit (4th order chebychev)

In [ ]:
#splot.SetUp().FactoryPDF("Polynomial::BG(Mdiphoton,{a0,a1[1,-100000,100000],a2[1,-100000,100000],a3[1,-100000,100000]})")
splot.SetUp().FactoryPDF("Polynomial::BG(Mdiphoton,{a0[0.1,-100,100],a1[0.1,-100,100],a2[0.1,-100,100],a3[0.1,-100,100]})")
splot.SetUp().LoadSpeciesPDF("BG",1)

Bin the data 6 times in {Mh}

In [ ]:
splot.Bins().LoadBinVar("Mdihadron",6,0.3,1.3)

Load data with TTree name and file location

Can try to suppress output in Binner.cpp

In [ ]:
splot.LoadData("tree_postprocess","/work/clas12/users/gmat/CLAS12Analysis/data/rg-a/june8.root")

In [ ]:
ROOT.Here.Go(splot)

In [ ]:
splot.MergeWeights()

In [ ]:
splot.saveWeightedTree(pwd+"/outSplot/DataWeightedTree.root")

Complete sPlots of the azimuthal angles

Setup fit manager

Load data and the sWeights from the above diphoton fits

In [ ]:
c=ROOT.TCanvas("c","c",1200,1200)
ROOT.gStyle.SetOptStat(0)
c.Divide(2,2)
hp=ROOT.TH2F("hp","Signal Helicity=+1;phi_h",100,-3.1415,3.1415,100,-3.1415,3.1415)
hm=ROOT.TH2F("hm","Signal Helicity=-1;phi_R",100,-3.1415,3.1415,100,-3.1415,3.1415)
hp_bg=ROOT.TH2F("hp_bg","BG Helicity=+1;phi_h",100,-3.1415,3.1415,100,-3.1415,3.1415)
hm_bg=ROOT.TH2F("hm_bg","BG Helicity=-1;phi_R",100,-3.1415,3.1415,100,-3.1415,3.1415)
splot.DrawWeighted("phi_h:phi_R>>hp","Signal","helicity==1","goff");
splot.DrawWeighted("phi_h:phi_R>>hm","Signal","helicity==-1","goff");
splot.DrawWeighted("phi_h:phi_R>>hp_bg","BG","helicity==1","goff");
splot.DrawWeighted("phi_h:phi_R>>hm_bg","BG","helicity==-1","goff");
c.cd(1)
hp.Draw("colz")
c.cd(2)
hm.Draw("colz")
c.cd(3)
hp_bg.Draw("colz")
c.cd(4)
hm_bg.Draw("colz")
c.Draw()

In [ ]:
c=ROOT.TCanvas("c","c",1200,1200)
#f=ROOT.TFile("../../../data/rg-a/june5.root","READ")
#f=ROOT.TFile("../../../data/rg-a/june5.root","READ")
#t=f.Get("tree_postprocess")
h1=ROOT.TH1F("h1","h1",100,0,0.4)
h2=ROOT.TH1F("h2","h2",100,0,0.4)
h3=ROOT.TH1F("h3","h3",100,0,0.4)
h1.SetLineColor(2)
h2.SetLineColor(1)
h3.SetLineColor(4)
splot.DrawWeighted("Mdiphoton>>h1","Signal","Mdihadron>0.3&&Mdihadron<0.8");
splot.DrawWeighted("Mdiphoton>>h2","BG","Mdihadron>0.3&&Mdihadron<0.8","same");
#t.Draw("Mdiphoton>>h3","","same")
c.Draw()

In [ ]:
f=ROOT.TFile("../../../data/rg-a/june5.root","READ")
t=f.Get("tree_postprocess")

In [ ]:
fm = ROOT.FitManager()
fm.SetUp().SetOutDir(pwd+"/outObs1/");
fm.Data().LoadWeights("Signal",pwd+"/outSplot/Tweights.root");

In [ ]:
f=ROOT.TFile("outSplot2/test.root","READ")
f.ls()

In [ ]:
t.Print()

In [ ]:
fm = ROOT.FitManager()